Task1_0820. Python REPL 도구를 사용하는 LangChain 예시를 수행하세요.

In [3]:
import sys
import io


class PythonREPL:
    def run(self, code: str) -> str:
        # 실행할 파이썬 코드의 출력값을 캡처
        old_stdout = sys.stdout
        new_stdout = io.StringIO()
        sys.stdout = new_stdout

        try:
            exec(code)  # 코드 실행
        except Exception as e:
            return str(e)
        finally:
            # 기존 stdout 복원
            sys.stdout = old_stdout

        # 실행 결과 반환
        return new_stdout.getvalue()


# 예시 사용
python_repl = PythonREPL()

# 실행할 코드 입력
code = """
a = 5
b = 10
print(a * b)
"""

# Python 코드 실행
result = python_repl.run(code)
print(result)

50



In [1]:
from openai import OpenAI
import os

MODEL = "gpt-4o-mini-2024-07-18"
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
import os
from uuid import uuid4

# unique_id를 uuid4()로 생성
unique_id = str(uuid4())

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"LangChain Tool Use Quickstart - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

In [6]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain import hub

llm = ChatOpenAI(model=MODEL)
python_repl = PythonREPL()

@tool
def run_code(code: str) -> str:
    """run a python code and return the output"""
    return python_repl.run(code)

tools = [run_code]
prompt = hub.pull("hwchase17/openai-tools-agent")

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke(
    {
        "input": """
        파이썬을 사용하여 25의 제곱한  값을 출력하시오.
        """
    }
)



> Entering new AgentExecutor chain...

Invoking: `run_code` with `{'code': 'result = 25 ** 2\nresult'}`


25의 제곱 값은 625입니다.

> Finished chain.


{'input': '\n        파이썬을 사용하여 25의 제곱한  값을 출력하시오.\n        ',
 'output': '25의 제곱 값은 625입니다.'}

# ex

initialize_agent vs. agent_executor
- initialize_agent는 에이전트를 초기화하고 실행하는 데 있어 간편하고 직관적인 방식을 제공하기 때문에, 복잡한 작업이 없고 간단한 대화형 에이전트를 만들 때는 더 적합합니다.
- agent_executor는 더 세밀한 제어가 필요할 때나, 이미 설정된 에이전트를 실행할 때 사용됩니다.

agent.invoke vs. agent.run
- LangChain 에이전트에서 사용하는 두 가지 메서드로, 각각의 동작 방식과 목적이 다릅니다.
- agent.run은 에이전트가 텍스트 입력을 받아 해당 입력에 대해 대화를 기반으로 작업을 수행할 때 사용되는 메서드입니다. 일반적으로 대화형 에이전트가 사용자의 명령이나 질문에 반응하는 방식으로 동작하며, 주로 다음과 같은 상황에서 사용됩니다:
  - 단일 사용자 입력 처리: 에이전트가 사용자로부터 한 번의 입력을 받아서 그 입력에 대한 적절한 응답을 생성하는 경우.
  - 대화형 응답: 에이전트가 주어진 입력에 대해 자동으로 도구(예: 파이썬 코드 실행 도구)를 사용하고, 그 결과를 사용하여 응답을 생성하는 구조.
  - 예시: response = agent.run("파이썬을 사용하여 25의 제곱을 계산해줘.")
- agent.invoke는 보다 세밀한 제어를 제공하는 메서드로, LangChain에서는 일반적으로 여러 개의 입력을 받을 때나 에이전트의 여러 기능을 조합해서 사용할 때 적합합니다. 이 메서드는 입력 메시지를 포함하는 포맷으로 전달하며, 이를 통해 대화나 실행 과정을 더 세밀하게 제어할 수 있습니다.
  - 메시지 기반 제어: invoke 메서드는 LangChain 메시지 포맷을 사용하여 좀 더 구조화된 입력을 받습니다. 이 메시지 포맷은 다양한 대화 구성 요소를 포함할 수 있기 때문에 복잡한 대화 흐름을 처리할 수 있습니다.
  - 다양한 입력 처리: 여러 종류의 입력을 동시에 처리하거나, 여러 도구를 연결하는 경우에 유리합니다.
  - 예시: response = agent.invoke([{"role": "user", "content": "파이썬을 사용하여 25의 제곱을 계산해줘."}])

In [14]:
from langchain.tools import tool
from langchain.agents import initialize_agent, Tool, AgentType, create_react_agent
from langchain_openai import ChatOpenAI
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

python_repl = PythonREPL()

tools = [
    Tool(
        name="PythonREPL",
        func=python_repl.run,
        description="Run Python code and return the output",
    )
]
# ChatOpenAI 모델을 사용하여 대화를 생성하는 에이전트 초기화
llm = ChatOpenAI(model=MODEL, temperature=0)
memory = ConversationBufferMemory(memory_key="chat_history")  #대화 기록을 위한 메모리 사용

agent = initialize_agent(
    tools,
    llm,
    agent_type=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
)

response = agent.run("파이썬을 사용하여 25의 제곱을 계산해줘")

print(response)



> Entering new AgentExecutor chain...
To calculate the square of 25 using Python, I will write a simple expression that multiplies 25 by itself. 

Action: PythonREPL
Action Input: "25 ** 2"
Observation: 
Thought:I need to check the result of the calculation to find the square of 25. 

Action: PythonREPL
Action Input: "25 ** 2"
Observation: 
Thought:I need to run the calculation again to get the square of 25.

Action: PythonREPL
Action Input: "25 ** 2"
Observation: 
Thought:I seem to be having trouble getting the output from the calculation. Let me try running the calculation again to ensure I get the correct result.

Action: PythonREPL
Action Input: "25 ** 2"
Observation: 
Thought:It seems there is an issue with retrieving the output. I will try to run the calculation one more time to ensure I get the correct result.

Action: PythonREPL
Action Input: "25 ** 2"
Observation: 
Thought:It seems that I am unable to retrieve the output from the calculation. I will attempt to run the calcul